In [184]:
#import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from io import StringIO
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

#불러온 모듈 webdriver에서 Chrom()객체를 생성


In [185]:
#링크 접속
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.melon.com/chart/index.htm")
# 창 크기 설정
# 요소를 찾고 클릭하는 동작에 대해 최대 3초 동안 대기
driver.implicitly_wait(3)

# 창 크기 가져오기 및 출력
size = driver.get_window_size()
width = size.get("width")
height = size.get("height")
print(f"{width}px {height}px")

# 창 크기 설정
driver.set_window_size(1400, 1346)




1050px 1000px


In [187]:
#차트 파인더
button = driver.find_element(By.XPATH, '//*[@id="gnb_menu"]/ul[1]/li[1]/div/div/button/span')
ActionChains(driver).click(button).perform()

#주간차트
button = driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/h4[1]')
ActionChains(driver).click(button).perform()
#연대
button = driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[1]/div[1]/ul/li[1]/span')
ActionChains(driver).click(button).perform()

#연도
button = driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[2]/div[1]/ul/li[1]')
ActionChains(driver).click(button).perform()

#월간

button = driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[3]/div[1]/ul/li[1]/span')
ActionChains(driver).click(button).perform()
#주간
button = driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[4]/div[1]/ul/li[1]/span')
ActionChains(driver).click(button).perform()
#장르
button = driver.find_element(By.XPATH, '//*[@id="d_chart_search"]/div/div/div[5]/div[1]/ul/li[1]/span')
ActionChains(driver).click(button).perform()

#버튼
button = driver.find_element(By.XPATH, '//*[@id="d_srch_form"]/div[2]/button/span')
ActionChains(driver).click(button).perform()


In [211]:
title =[]
album = []
singer = []
date = []
like = []
rank = []

In [212]:
print(len(title))
print(len(singer))
print(len(album))
print(len(date))
print(len(like))
print(len(rank))

0
0
0
0
0
0


In [209]:
# 각자 가져오는 방법, 이거부터 실행
driver.implicitly_wait(3)
html_source = driver.page_source

parse = BeautifulSoup(html_source, 'html.parser')

titles = parse.find_all("div", {"class": "ellipsis rank01"}) 
singers = parse.find_all("div", {"class": "ellipsis rank02"}) 
albums = parse.find_all("div",{"class": "ellipsis rank03"}) 
dates = driver.find_element(By.CLASS_NAME,'datelk').text
likes_elements = parse.find_all("span", {"class": "cnt"})  # 좋아요 수 추출

likes = [element.text.strip() for element in likes_elements]


rank = [i for i in range(1, len(title) + 1)]


for t in titles:
    title.append(t.find('a').text) 
for s in singers:
    singer.append(s.find('span', {"class": "checkEllipsis"}).text)

for a in albums:
    album.append(a.find('a').text)
    date.append(dates)
for l in likes:
    l = (re.sub(r'\D', '', l.replace("총건수", "").strip()))
    like.append(l)

rank = [i for i in range(1, len(title) + 1)]
#for i in range(len(rank)):
    #print('%s위: %s [ %s ] - %s'%(rank[i], title[i], album[i], singer[i]),date[i],like[i])
print(len(title),
        len(singer),
        len(album),
        len(like),
        len(date),
        len(rank))
#print(title[-1], album[-1], singer[-1],date[-1],like[-1])

198 198 198 198 198 99


In [128]:
data = {
        'rank' : rank * (len(title) // 100),
        'title': title,
        'singer': singer,
        'album': album,
        'dates': date,
        'like' : like, 
        
    }

chart_df = pd.DataFrame(data)
chart_df.to_csv('melon_chart.csv', index=False, encoding='utf-8-sig')

In [167]:
# 년도별 가장 많이

# Updated 'desired_months' to include January
desired_months = ["2024.03", "2023.03", "2022.03",
                "2024.04", "2023.04", "2022.04",
                "2024.05", "2023.05", "2022.05"]

# 'dates' 열에서 연월 추출
chart_df['year_month'] = chart_df['dates'].str.extract(r'(\d{4}.\d{2})')

# 원하는 연월에 해당하는 행만 필터링하여 새 데이터프레임 생성
chart_df_filter = chart_df[chart_df['year_month'].isin(desired_months)].reset_index(drop=True)


chart_df_filter
#chart_df_filter.to_csv('melon_chart_filter_test.csv', index=False, encoding='utf-8-sig')

chart_df_filter['dates'].str.slice(0, 5) 


,rank,title,singer,album,dates,like,year_month
0,1,Ditto,NewJeans,NewJeans 'OMG',2023.03.06 ~ 2023.03.12,281544,2023.03
1,2,OMG,NewJeans,NewJeans 'OMG',2023.03.06 ~ 2023.03.12,200692,2023.03
2,3,Hype Boy,NewJeans,NewJeans 1st EP 'New Jeans',2023.03.06 ~ 2023.03.12,287850,2023.03
3,4,Teddy Bear,STAYC(스테이씨),Teddy Bear,2023.03.06 ~ 2023.03.12,109914,2023.03
4,5,사건의 지평선,윤하 (YOUNHA),YOUNHA 6th Album Repackage 'END THEORY : Final...,2023.03.06 ~ 2023.03.12,322104,2023.03
...,...,...,...,...,...,...,...
2595,96,연애편지,임영웅,IM HERO,2022.05.30 ~ 2022.06.05,52187,2022.05
2596,97,Real Love,오마이걸 (OH MY GIRL),Real Love,2022.05.30 ~ 2022.06.05,46061,2022.05
2597,98,사랑하긴 했었나요 스쳐가는 인연이었나요 짧지않은 우리 함께했던 시간들이 자꾸 내 마...,잔나비,봉춤을 추네,2022.05.30 ~ 2022.06.05,200068,2022.05
2598,99,DON QUIXOTE,세븐틴 (SEVENTEEN),SEVENTEEN 4th Album 'Face the Sun',2022.05.30 ~ 2022.06.05,61512,2022.05


In [176]:
chart_df_filter['dates'] = chart_df_filter['dates'].str.slice(0, 4) 



IndexError: Column(s) title already selected